In [1]:
#point to local path for lotus package directory
import sys
sys.path.append(r'C:\Users\Kingdel\Documents\GitHub')

from lotus import * #bulk import statement to import standard and custom libraries

Loading BokehJS ...

In [2]:
board_num=0
bit_num=0 
bit_val=1 #--> normally high, but manually set to 1
analog_channel=0

mc_init(board_num) #initialize the mc DAQ 
mc_digital_out(board_num,bit_num,bit_val) ##set mcc 5V to HIGH, power the sensor

#initialize mc analog input
ai_props = AnalogInputProps(board_num)
ai_range = ai_props.available_ranges[0]

#dmm serial port setup
ser=serial.Serial('COM3',timeout=3.0)
ser.flushInput()

#experimentally determined gain factor
lpt_gain=0.000595 #inches per ohm

Found device: USB-1808X (1D69D8B)

Setting AUXPORT to 255.
Setting AUXPORT bit 0 to 1.


In [3]:
interrupt=False #variable to control data loop
data=pd.DataFrame([],columns=['posn(in)','output(V)']) #blank dataframe to hold data

#setup plot
p1=figure(plot_width=650,plot_height=350)
t=show(p1,notebook_handle=True)

button=widgets.Button(description='start data logging')
display(button)

def when_clicked(b): #ugly hack to get real-time datalogging
    def get_data():
        global interrupt
        global data
        while not interrupt:     
                
                #potentiometer resistance
                cmd='READ?'
                res=dmm_cmd(cmd,ser,0.1)
                res=float(res.strip()) 
                x=res*lpt_gain
                
                # sensor voltage
#                 value = ul.a_in(board_num, analog_channel, ai_range) ##uncomment for 12-bit board USB-202
#                 y = ul.to_eng_units(board_num, ai_range, value) 
                
                value = ul.a_in_32(board_num, analog_channel, ai_range) ##uncomment for high-res board USB-1808
                y = ul.to_eng_units_32(board_num, ai_range, value) 
                              
                data.loc[len(data)]=[x,y]           
                r1=p1.circle(x,y,size=7)
                push_notebook(handle=t)

                time.sleep(0.05)
    thread = threading.Thread(target=get_data)
    thread.start()

button.on_click(when_clicked)


Button(description='start data logging', style=ButtonStyle())

In [5]:
output_dir=r'C:\Users\Kingdel\Desktop\data'
os.chdir(output_dir)

filename='TMR2901_3b.csv'

In [6]:
button2=widgets.Button(description='stop data logging')
display(button2)
def stop_thread(c):
    global interrupt
    interrupt=True
    
    data.to_csv(filename)
    mc_digital_out(board_num,bit_num,0) ##set mcc 5V to LOW, power OFF the sensor
    
button2.on_click(stop_thread)

Button(description='stop data logging', style=ButtonStyle())

Setting AUXPORT to 255.
Setting AUXPORT bit 0 to 0.


In [7]:
# mc_init(board_num) #initialize the mc DAQ 
# mc_digital_out(board_num,bit_num,0) ##set mcc 5V to LOW, power off the sensor